In [1]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [2]:
# 1. Carregar os dados
arquivo = "dados_limpos.csv"
dados = pd.read_csv(arquivo)

In [3]:
# 2. Organizar os produtos por transação
compras = dados.groupby("id_da_compra")["produto"].agg(list)

In [4]:
# 3. Filtrar compras com mais de um item
compras_validas = compras[compras.apply(lambda itens: len(set(itens)) > 1)]

In [5]:
# 4. Simular dados caso não haja transações suficientes
if compras_validas.empty:
    print("Poucas transações válidas. Gerando dados simulados...")
    simulado = dados.sample(frac=1).reset_index(drop=True)
    simulado["compra_id_fake"] = simulado.index // 5
    compras_validas = simulado.groupby("compra_id_fake")["produto"].agg(list)

Poucas transações válidas. Gerando dados simulados...


In [6]:
# 5. Transformar dados em formato binário (one-hot)
encoder = TransactionEncoder()
one_hot = encoder.fit_transform(compras_validas)
df_binario = pd.DataFrame(one_hot, columns=encoder.columns_)

In [7]:
# 6. Descobrir itens frequentes com FP-Growth
frequentes = fpgrowth(df_binario, min_support=0.05, use_colnames=True)

In [8]:
# 7. Verificar se há itens frequentes
if frequentes.empty:
    print("Nenhum item frequente foi encontrado. Tente reduzir o min_support.")
else:
    # 8. Gerar regras de associação com confiança mínima de 20%
    regras = association_rules(frequentes, metric="confidence", min_threshold=0.2)

    # 9. Filtrar regras com qualidade (lift > 1 e confiança >= 0.2)
    regras_filtradas = regras[["antecedents", "consequents", "support", "confidence", "lift"]]
    regras_filtradas = regras_filtradas[
        (regras_filtradas["lift"] > 1) & (regras_filtradas["confidence"] >= 0.2)
    ].sort_values(by="lift", ascending=False)

    # 10. Exibir resumo das regras encontradas
    print(f"Total de regras relevantes encontradas: {len(regras_filtradas)}")
    print(regras_filtradas.head(10))

    # 11. Exportar as regras para um arquivo CSV
    regras_filtradas.to_csv("regras_associacao.csv", index=False)
    print("As regras foram salvas em 'regras_associacao.csv'.")


Total de regras relevantes encontradas: 1080
                                 antecedents  \
1079                         (condicionador)   
0                                 (sabonete)   
1                                 (presunto)   
2                           (leite integral)   
3                                 (presunto)   
4                           (leite integral)   
5                                 (sabonete)   
6                 (leite integral, sabonete)   
1063  (shampoo, condicionador, refrigerante)   
1062   (shampoo, condicionador, sabao em po)   

                                            consequents   support  confidence  \
1079  (pasta de dente, shampoo, refrigerante, sabao ...  0.142857         1.0   
0                                            (presunto)  0.142857         1.0   
1                                            (sabonete)  0.142857         1.0   
2                                            (presunto)  0.142857         1.0   
3                    

In [9]:
#Combos frequentes de produtos (pares mais comuns)
# Filtrar apenas regras com 2 itens (par de produtos)
regras_pares = regras_filtradas[
    (regras_filtradas["antecedents"].apply(len) == 1) &
    (regras_filtradas["consequents"].apply(len) == 1)
]

# Criar string com o combo e mostrar top 10
regras_pares["combo"] = regras_pares.apply(
    lambda row: f"{list(row['antecedents'])[0]} → {list(row['consequents'])[0]}", axis=1
)

top_combos = regras_pares[["combo", "support", "confidence", "lift"]].sort_values(by="lift", ascending=False).head(10)

print("\nTop 10 Combos de Produtos (por Lift):")
print(top_combos)

top_combos.to_csv("combos_top10.csv", index=False)
print("Arquivo 'combos_top10.csv' salvo com sucesso!")



Top 10 Combos de Produtos (por Lift):
                        combo   support  confidence  lift
0         sabonete → presunto  0.142857         1.0   7.0
1         presunto → sabonete  0.142857         1.0   7.0
2   leite integral → presunto  0.142857         1.0   7.0
3   presunto → leite integral  0.142857         1.0   7.0
4   leite integral → sabonete  0.142857         1.0   7.0
5   sabonete → leite integral  0.142857         1.0   7.0
55            presunto → cafe  0.142857         1.0   7.0
54            cafe → presunto  0.142857         1.0   7.0
53      leite integral → cafe  0.142857         1.0   7.0
52      cafe → leite integral  0.142857         1.0   7.0
Arquivo 'combos_top10.csv' salvo com sucesso!


C:\Users\Breno\AppData\Local\Temp\ipykernel_2852\1351044807.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regras_pares["combo"] = regras_pares.apply(


In [10]:
#Produtos com maior Lift médio em regras
# Criar lista com todos produtos envolvidos e seus lifts
lift_por_produto = []

for _, linha in regras_filtradas.iterrows():
    for item in linha["antecedents"].union(linha["consequents"]):
        lift_por_produto.append((item, linha["lift"]))

# Calcular lift médio por produto
df_lift = pd.DataFrame(lift_por_produto, columns=["produto", "lift"])
media_lift = df_lift.groupby("produto")["lift"].mean().sort_values(ascending=False).head(10)

# Mostrar resultado
print("\nProdutos com maior lift médio:")
print(media_lift)

media_lift.to_csv("produtos_lift_medio.csv", header=["lift_medio"])
print("Arquivo 'produtos_lift_medio.csv' salvo com sucesso!")



Produtos com maior lift médio:
produto
acucar           7.0
agua mineral     7.0
amaciante        7.0
arroz            7.0
cafe             7.0
carvao           7.0
cerveja          7.0
condicionador    7.0
desinfetante     7.0
detergente       7.0
Name: lift, dtype: float64
Arquivo 'produtos_lift_medio.csv' salvo com sucesso!
